# Integration of PyGSTi (gate set tomography) with Mitiq (PEC)
The purpose of this notebook is to use the data collected from a gate set tomography (GST) experiment that has been saved into the `rigetti_first_run` directory and perform GST on it. 

The tomographic results are used to apply error mitigation via the probabilistic error cancellation (PEC) module of Mitiq.

This code was mostly taken from the tutorial notebook [UnitaryFundDemoNotebook1.ipynb
](https://zenodo.org/record/5715199#.Yr_TOUjMLJF). 

In [29]:
# PyGSTi tools
import pygsti
from pygsti.modelpacks import smq1Q_XZ

# Mitiq tools
from mitiq.pec import NoisyOperation, NoisyBasis, execute_with_pec
from mitiq.pec.representations.optimal import find_optimal_representation

import numpy as np

# Qiskit tools for mapping Pauli transfer matrices to superoperators.
from qiskit.quantum_info import PTM, SuperOp

# Cirq library used as a frontend for interfacing Mitiq and for simulations
import cirq

## Settings

If `RUN_GST_IMPOSING_NOISELESS_RZ` is `False`, standard GST is performed without constraints.

If `RUN_GST_IMPOSING_NOISELESS_RZ` is `True`, GST is performed assuming `Rz` rotations are noiseless.

In [57]:
RUN_GST_IMPOSING_NOISELESS_RZ = False

# Make numpy matrices easier to read
np.set_printoptions(precision=3, suppress=True)

## Read data from experiment
We load data that was previously created by the `get_gst_data.ipynb` notebook.

In [31]:
data = pygsti.io.read_data_from_dir('experiment_data/rigetti_first_run')

## Run gate set tomography (GST)


We apply GST to the raw experimental data to obtain the Pauli transfer matrices associated to the gates $R_X(\pi/2)$ and $R_Z(\pi/2)$.

If `RUN_GST_IMPOSING_NOISELESS_RZ` is `True` we also impose Rz rotations to be noiseless by following a procedure adapted from page 3 of https://arxiv.org/abs/2002.12476.

In [32]:
if not RUN_GST_IMPOSING_NOISELESS_RZ:
    gst_protocol = pygsti.protocols.StandardGST('CPTP')
    # Run GST
    results = gst_protocol.run(data)
    # Get model
    model = results.estimates['CPTP'].models['stdgaugeopt']
else:
    # Define a noiseless Rz(pi/2) operator.
    initial_model = smq1Q_XZ.target_model()
    initial_model.set_all_parameterizations("CPTP")
    rz_matrix = initial_model[('Gzpi2',0)].to_dense()
    dense_rz = pygsti.modelmembers.operations.DenseOperator(rz_matrix, evotype="default")
    # Enforce a noisless Rz(pi/2) gate.
    initial_model[('Gzpi2',0)] = dense_rz.copy()
    # Run GST
    results = pygsti.protocols.GST(initial_model, gaugeopt_suite=None).run(data)
    # Get model
    model = results.estimates['GateSetTomography'].models['final iteration estimate']

-- Std Practice:  Iter 1 of 1  (CPTP) --: 


  --- Iterative GST: [##################################################] 100.0%  550 circuits ---
  Iterative GST Total Time: 21.4s


In [33]:
print("Rx Pauli transfer matrix:\n", model.operations[('Gxpi2', 0)].to_dense())
print("Rz Pauli transfer matrix:\n", model.operations[('Gzpi2', 0)].to_dense())

Rx Pauli transfer matrix:
 [[ 1.     0.     0.     0.   ]
 [ 0.007  0.991  0.033 -0.034]
 [-0.002 -0.027  0.013 -0.977]
 [-0.003 -0.029  0.976  0.011]]
Rz Pauli transfer matrix:
 [[ 1.     0.     0.     0.   ]
 [ 0.    -0.    -0.996 -0.031]
 [-0.     1.    -0.001  0.031]
 [ 0.    -0.031 -0.031  0.996]]


## Get noisy superoperators

We convert the estimated Pauli transfer matrices into superoperator matrices. Superoperator matrices are equivalent representations of the noisy gates which are more suitable for applying PEC with Mitiq.

In [35]:
# Convert Pauli transfer matrix to superoperator
superop_rx = SuperOp(PTM(model.operations[('Gxpi2', 0)].to_dense())).data.conj()  # conj depends on convention
superop_rz = SuperOp(PTM(model.operations[('Gzpi2', 0)].to_dense())).data.conj()  # conj depends on convention

print("Noisy Rx superoperator matrix:\n", superop_rx)
print("Noisu Rz superoperator matrix:\n", superop_rz)


Rx Pauli transfer matrix:
 [[ 0.504-0.j    -0.015+0.488j -0.015-0.488j  0.493-0.j   ]
 [-0.014+0.489j  0.502+0.03j   0.489-0.003j  0.021-0.487j]
 [-0.014-0.489j  0.489+0.003j  0.502-0.03j   0.021+0.487j]
 [ 0.496-0.j     0.015-0.488j  0.015+0.488j  0.507-0.j   ]]
Rz Pauli transfer matrix:
 [[ 0.998-0.j    -0.015-0.015j -0.015+0.015j  0.002-0.j   ]
 [-0.015-0.015j -0.001-0.998j  0.   -0.002j  0.015+0.015j]
 [-0.015+0.015j  0.   +0.002j -0.001+0.998j  0.015-0.015j]
 [ 0.002-0.j     0.015+0.015j  0.015-0.015j  0.998-0.j   ]]


## Get ideal superoperators.

We also define the superoperator matrices corresponding to the ideal gates.

In [39]:
# Create ideal operations as Cirq circuits
cirq_rx = cirq.Circuit(cirq.rx(np.pi / 2).on(cirq.LineQubit(0)))
cirq_rz = cirq.Circuit(cirq.rz(np.pi / 2).on(cirq.LineQubit(0)))

# Get ideal superoperators
superop_rx_cirq = cirq.kraus_to_superoperator(cirq.kraus(cirq_rx))
superop_rz_cirq = cirq.kraus_to_superoperator(cirq.kraus(cirq_rz))

print("Noisless Rx superoperator matrix:\n", superop_rx_cirq)
print("Noisless Rz superoperator matrix:\n", superop_rz_cirq)

Noisless Rx superoperator matrix:
 [[0.5+0.j  0. +0.5j 0. -0.5j 0.5+0.j ]
 [0. +0.5j 0.5+0.j  0.5+0.j  0. -0.5j]
 [0. -0.5j 0.5+0.j  0.5+0.j  0. +0.5j]
 [0.5+0.j  0. -0.5j 0. +0.5j 0.5+0.j ]]
Noisless Rz superoperator matrix:
 [[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.-1.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+1.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j]]


## Define the basis of noisy operations

We now define a `mitiq.pec.NoisyBasis`, i.e. a basis of noisy gate sequences that will be used to represent ideal operations in PEC.

In [40]:
# Create noisy operations as Mitiq NoisyOperation objects
mitiq_noisy_rx = NoisyOperation(cirq_rx, superop_rx)
mitiq_noisy_rz = NoisyOperation(cirq_rz, superop_rz)

# Combine into basis object
base_noisy_basis = NoisyBasis(mitiq_noisy_rx, mitiq_noisy_rz)

# Extend basis to squences of noisy gates
noisy_basis = NoisyBasis(
    *base_noisy_basis.get_sequences(1),
    *base_noisy_basis.get_sequences(2),
    *base_noisy_basis.get_sequences(3),
    *base_noisy_basis.get_sequences(4),
    # *base_noisy_basis.get_sequences(5),   
    # *base_noisy_basis.get_sequences(6),   
)

print("Sequeces of noisy operations to be used as a basis for PEC:")
for elem in noisy_basis.elements:
    print(elem)

Sequeces of noisy operations to be used as a basis for PEC:
0: ───Rx(0.5π)───Rx(0.5π)───
0: ───Rx(0.5π)───Rz(0.5π)───Rx(0.5π)───
0: ───Rx(0.5π)───Rx(0.5π)───Rz(0.5π)───
0: ───Rx(0.5π)───Rx(0.5π)───Rx(0.5π)───
0: ───Rx(0.5π)───Rz(0.5π)───Rz(0.5π)───
0: ───Rx(0.5π)───Rz(0.5π)───Rz(0.5π)───Rx(0.5π)───
0: ───Rx(0.5π)───Rx(0.5π)───Rz(0.5π)───Rz(0.5π)───
0: ───Rz(0.5π)───Rz(0.5π)───
0: ───Rx(0.5π)───
0: ───Rz(0.5π)───Rx(0.5π)───Rx(0.5π)───
0: ───Rz(0.5π)───Rz(0.5π)───Rx(0.5π)───Rx(0.5π)───
0: ───Rx(0.5π)───Rx(0.5π)───Rz(0.5π)───Rx(0.5π)───
0: ───Rx(0.5π)───Rz(0.5π)───Rx(0.5π)───Rx(0.5π)───
0: ───Rz(0.5π)───Rz(0.5π)───Rz(0.5π)───Rx(0.5π)───
0: ───Rx(0.5π)───Rz(0.5π)───Rz(0.5π)───Rz(0.5π)───
0: ───Rz(0.5π)───Rz(0.5π)───Rz(0.5π)───Rz(0.5π)───
0: ───Rz(0.5π)───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)───
0: ───Rz(0.5π)───Rx(0.5π)───
0: ───Rz(0.5π)───Rz(0.5π)───Rz(0.5π)───
0: ───Rx(0.5π)───Rx(0.5π)───Rx(0.5π)───Rz(0.5π)───
0: ───Rx(0.5π)───Rx(0.5π)───Rx(0.5π)───Rx(0.5π)───
0: ───Rz(0.5π)───Rx(0.5π)───Rz(0.5

## Compute optimal representation

We now use Mitiq to obtain an optimal quasi-probability representation of the noiseless $R_x(\pi/2)$ as a linear combination of the noisy operations printed above.

In [46]:
# Get a representation with minimum PEC overhead parameter.
optimal_rep = find_optimal_representation(cirq_rx, noisy_basis, tol=.001)

print(optimal_rep)
print("\nPEC overhead parameter: ", optimal_rep.norm) # Hopefully close to 1

0: ───Rx(0.5π)─── = 0.035*(0: ───Rx(0.5π)───Rz(0.5π)───Rx(0.5π)───)-0.013*(0: ───Rx(0.5π)───Rx(0.5π)───Rx(0.5π)───)-0.017*(0: ───Rx(0.5π)───Rz(0.5π)───Rz(0.5π)───Rx(0.5π)───)+0.003*(0: ───Rx(0.5π)───Rx(0.5π)───Rz(0.5π)───Rz(0.5π)───)+1.002*(0: ───Rx(0.5π)───)+0.058*(0: ───Rx(0.5π)───Rx(0.5π)───Rz(0.5π)───Rx(0.5π)───)-0.004*(0: ───Rz(0.5π)───Rz(0.5π)───Rz(0.5π)───Rx(0.5π)───)+0.190*(0: ───Rz(0.5π)───Rz(0.5π)───Rz(0.5π)───Rz(0.5π)───)-0.206*(0: ───Rx(0.5π)───Rx(0.5π)───Rx(0.5π)───Rx(0.5π)───)-0.004*(0: ───Rz(0.5π)───Rz(0.5π)───Rx(0.5π)───)+0.019*(0: ───Rz(0.5π)───)-0.063*(0: ───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)───Rz(0.5π)───)

PEC overhead parameter:  1.6131640702970076


Let's verify that the liner combination of noisy channels defined by `optimal_rep` reproduces the ideal  channel $R_x(\pi/2)$ as expected.

In [56]:
basis_matrices = [m.channel_matrix for m in optimal_rep.noisy_operations]
represented_rx = sum(m * coeff for (m, coeff) in zip(basis_matrices, optimal_rep.coeffs))
print("Approximated Rx superoperator:")
print(represented_rx)
print("\nIdeal Rx superoperator:")
print(superop_rx_cirq)

Approximated Rx superoperator:
[[ 0.499+0.j     0.001+0.499j  0.001-0.499j  0.499-0.j   ]
 [ 0.001+0.501j  0.499+0.001j  0.499-0.001j  0.001-0.499j]
 [ 0.001-0.501j  0.499+0.001j  0.499-0.001j  0.001+0.499j]
 [ 0.501-0.j    -0.001-0.499j -0.001+0.499j  0.501+0.j   ]]

Ideal Rx superoperator:
[[0.5+0.j  0. +0.5j 0. -0.5j 0.5+0.j ]
 [0. +0.5j 0.5+0.j  0.5+0.j  0. -0.5j]
 [0. -0.5j 0.5+0.j  0.5+0.j  0. +0.5j]
 [0.5+0.j  0. -0.5j 0. +0.5j 0.5+0.j ]]


## Define a Mitiq executor

In order to apply PEC with Mitiq we need to define an executor, i.e. a function mapping circuits to expectation values. Actually, in this example, we define two executors corresponding to two different simulation methods:

- `cirq_executor` uses Cirq to simulate noisy circuits assuming perfect measurements, such that the effect of PEC is more evident.
- `pygsti_executor` uses PyGSTi to simulate noisy circuits assuming noisy measurements. This executor provides a more accurate simulation of the real hardware.

In [12]:
# Helper functions

def ideal_to_noisy_circuit(ideal_circuit):
    noisy_rx = cirq.KrausChannel(cirq.superoperator_to_kraus(superop_rx)).on(cirq.LineQubit(0))
    noisy_rz = cirq.KrausChannel(cirq.superoperator_to_kraus(superop_rz)).on(cirq.LineQubit(0))

    noisy_circuit = cirq.Circuit()
    for op in ideal_circuit.all_operations():
        if op.gate == cirq.rx(np.pi / 2):
            noisy_circuit.append(noisy_rx)
        elif op.gate == cirq.rz(np.pi / 2):
            noisy_circuit.append(noisy_rz)
        else:
            print("Error:", op)

    return noisy_circuit

def cirq_to_pygsti(ideal_circuit: cirq.Circuit):
    pygsti_ops = []
    for op in ideal_circuit.all_operations(): 
        if op.gate == cirq.rx(np.pi / 2):
            pygsti_ops.append(('Gxpi2',0))
        elif op.gate == cirq.rz(np.pi / 2):
            pygsti_ops.append(('Gzpi2',0))
        else:
            print("Error: impossible to simulate operation", op)

    return pygsti.circuits.Circuit(pygsti_ops, line_labels=[0])

In [64]:
# Executors returning the expectation value of of the observable A = |1><1|

def cirq_executor(circuit):
    """Returns the expectation value of the observable A = |1><1|"""
    noisy_circuit = ideal_to_noisy_circuit(circuit)
    rho = cirq.DensityMatrixSimulator().simulate(noisy_circuit).final_density_matrix
    return rho[1, 1].real

def pygsti_executor(circuit):
    """Returns the expectation value of the observable A = |1><1|"""
    noisy_circuit = cirq_to_pygsti(circuit)
    probs = model.probabilities(noisy_circuit)
    return probs["1"]

## Define circuit to mitigate

## Estimate an expectation value without error mitigation

We first define a simple circuit of depth 2, corresponding to the bit flip unitary $X = R_X(\pi/2)^2$. The ideal expectation value of the test observable $A=|1\rangle\langle 1|$ is equal to 1. 

In [63]:
bit_flip_circuit =  2 * cirq_rx
bit_flip_circuit

0: ───Rx(0.5π)───Rx(0.5π)───

The noisy unmitigated expectation value can be obtained by calling the noisy executor(s).

In [16]:
# Cirq simulation without measurement errors
cirq_executor(bit_flip_circuit)

0.9787803

In [17]:
# PyGSTi simulation with measurement errors
pygsti_executor(bit_flip_circuit)

0.8251257027477169

## Apply PEC

In [18]:
# PEC without measurement errors
execute_with_pec(bit_flip_circuit, cirq_executor, representations=(optimal_rep, ), random_state=0, num_samples=10000)

0.9919102353241187

In [19]:
# PEC with measurement errors
execute_with_pec(bit_flip_circuit, pygsti_executor, representations=(optimal_rep, ), random_state=0, num_samples=10000)

0.8358617831940299

## Conclusions

From the above results we can see that PEC improved the estimation of the expectation value for both simulation methods.
However, in the presence of measurement errors, the effect of PEC is less evident. This is not surprising since the PEC method applied in this tutorial only mitigates gate errors. 

In principle PEC could be extended in such a way to mitigate measurement errors by including the noisy POVMs obtained from GST in the basis of noisy operators for PEC.  This is however beyond the scope of this tutorial and, moreover, this is currently not supported in Mitiq.

We also note that instead of applying PEC for mitigating measurement errors, one could also apply other  readout error mitigation techniques like, e.g., those in the ` mitiq.rem` module.